In [1]:
import numpy as np
import pandas as pd
import re
import string
from keras.models import Sequential
from keras.layers import Dense
import numpy
from sklearn import preprocessing
from keras import optimizers
from matplotlib import pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression

Using TensorFlow backend.


# Transform all to numbers

In [2]:
#read data
fpt=pd.DataFrame.from_csv('FPT2.csv')
tgdd=pd.DataFrame.from_csv('thegioididong2.csv')
train_df=tgdd.append(fpt,ignore_index=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)
#remove the dots in 'Price' column
a=train_df['Price']
a.update(a[a.notnull()].map(lambda x: re.sub('\.','',str(x))))
#train_df['Price'].astype(float)

In [3]:
train_df

,Phone Name,resolution,screen size,rear camera,front camera,CPU,RAM,Internal storage,Battery capacity,Days,Price
0,OPPO F3,1080 x 1920,5.5 inch,13.0 MP,16.0 MP,1.51 Ghz,4 GB,64 GB,3200 mAh,1.0,6990000
1,Samsung Galaxy J5 Prime,720 x 1280,"5""",13 MP,5 MP,1.4 GHz,2 GB,16 GB,2400 mAh,240.0,4490000
2,Huawei GR5,1080 x 1920,5.5,13 MP,5 MP,1.5 GHz,2 GB,16 GB,3000 mAh,365.0,3990000
3,Samsung Galaxy C9 Pro,1080 x 1920,"6""",16 MP,16 MP,1.95 GHz,6 GB,64 GB,4000 mAh,180.0,11490000
4,Samsung Galaxy A3 (2017),1280 x 720,4.7 inch,13.0 MP,8.0 MP,1.6 GHz,2 GB,16 GB,2350 mAh,134.0,6490000
5,Itel it5020,320 x 240,2.4,2 MP,NaN,NaN,NaN,NaN,1000 mAh,180.0,300000
6,Wing K6,240 x 320,2.4 inch,NaN,NaN,NaN,NaN,NaN,800 mAh,174.0,429000
7,OPPO A37 (A37fw),720 x 1280,"5""",8 MP,5 MP,1.2 GHz,2 GB,16 GB,2630 mAh,NaN,3290000
8,Samsung Galaxy A7 (2016),1080 x 1920,5.5,13 MP,5 MP,1.6 GHz,3 GB,16 GB,3300 mAh,365.0,8990000
9,Philips E181,320 x 240,2.4,NaN,NaN,NaN,NaN,NaN,3100 mAh,365.0,790000


In [4]:
def refine_brands(df_train):#keep brands named below and set the other brands as 'other'
    df=df_train.copy()
    names=df['Phone Name']
    names.update(names.map(lambda x: x.split()[0]))
    con=(names=='iPhone') | (names=='Sony') | (names=='Samsung') | (names=='Oppo')
    names.update(names[~con].map(lambda x : 'other'))
    return df
df_train=refine_brands(train_df)

In [5]:
def one_hot(column):
    set_=list(set(column.values))
    set_=sorted(set_)
    length=len(column.values)
    value=column.values
    columns=np.zeros((length,len(set_)))
    for i in range(length):
        pos=set_.index(value[i])
        columns[i,pos]=1
    return set_,columns

def combine(board,label,column):
    for i in range(len(label)-1):
        board[label[i]]=column[:,i]
    return board
def process_categorical_cols(input_df):
    # TODO
    numeric_input_df=input_df.copy()
    price=numeric_input_df['Phone Name']
    numeric_input_df.drop(['Phone Name'],axis=1,inplace=True)
    
    one_hot_set,one_hot_column=one_hot(price)
    numeric_input_df=combine(numeric_input_df,one_hot_set,one_hot_column)
    
    return numeric_input_df
train_df=process_categorical_cols(df_train)

In [6]:
def resolution_process(df_train):#convert the from AxB to one number
    temp=df_train.copy()
    resolution=temp['resolution']
    temp.drop('resolution',axis=1)
    
    con=resolution.notnull()
    resolution.update(resolution[con].map(lambda x: int(x.split()[0])*int(x.split()[-1])))
    
    temp.update(resolution)
    return temp
train_df=resolution_process(train_df)

In [7]:
def num_filter(series):#remove characters which are not numbers
    con=series.notnull()
    series.update(series[con].map(lambda x: re.sub('[^0-9.]+','',x)))
    return series
def df_num_filter(train_df):#completely process every columns named below
    temp=train_df.copy()
    temp.update(num_filter(temp['screen size'].copy()))
    temp.update(num_filter(temp['rear camera'].copy()))
    temp.update(num_filter(temp['front camera'].copy()))
    temp.update(num_filter(temp['CPU'].copy()))
    temp.update(num_filter(temp['RAM'].copy()))
    temp.update(num_filter(temp['Internal storage'].copy()))
    temp.update(num_filter(temp['Battery capacity'].copy()))
    return temp
train_df=df_num_filter(train_df)

# Preprocessing

In [8]:
#devide total one into val, test and train dataset
val_df = train_df.iloc[:60, :].reset_index(drop=True)
train_df=train_df.iloc[60:, :].reset_index(drop=True)

test_df = train_df.iloc[:60, :].reset_index(drop=True)
train_df=train_df.iloc[60:, :].reset_index(drop=True)

In [9]:
#fill in the blank in the 'price' column with average value
def complete_price(df_train):#this is used for training dataset
    df=df_train.copy()
    column=df['Price']
    null_column=column.isnull()
    nonnull_column=column.notnull()
    mean=column[nonnull_column].map(lambda x: re.sub('\.','',x)).astype(float).mean()
    column.update(column[null_column].map(lambda x: mean))
    return df
train_df.update(complete_price(train_df))

In [10]:
#the same as above
def mean_fill(df_train,string):#this is used for training dataset
    df=df_train.copy()
    column=df[string]
    null_column=column.isnull()
    nonnull_column=column.notnull()
    mean=column[nonnull_column].astype(float).mean()
    column.update(column[null_column].map(lambda x: mean))
    return df
train_df.update(mean_fill(train_df,'Days'))
train_df.update(mean_fill(train_df,'screen size'))
train_df.update(mean_fill(train_df,'resolution'))
train_df.update(mean_fill(train_df,'Battery capacity'))

In [11]:
def zero_fill(df_,string):#fill in the deficient value with zero
    df=df_.copy()
    col=df[string]
    null_col=col.isnull()
    col[null_col]=0.0
    temp=col.copy().astype(float)
    df.drop(string,axis=1, inplace=True)
    df[string]=temp.values.astype(float)
    #df.update(col.astype(float))
    return df
def complete_zero_fill(df):#
    df.update(zero_fill(df,'rear camera'))
    df.update(zero_fill(df,'front camera'))
    df.update(zero_fill(df,'RAM'))
    df.update(zero_fill(df,'CPU'))
    df.update(zero_fill(df,'Internal storage'))
complete_zero_fill(train_df)
complete_zero_fill(val_df)
complete_zero_fill(test_df)

C:\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
def mean(train_df,string):
    col=train_df[string]
    if string=='Price':
        col=col.map(lambda x: re.sub('\.','',str(x)))
    return col.astype(float).mean()
def fill_mean(train_df,df,string):
    con=df[string].isnull()
    df[con]=mean(train_df,string)
    return df
def full_fill(train_df,df):#this is used for val and test dataset
    df.update(zero_fill(train_df,'rear camera'))
    df.update(zero_fill(train_df,'front camera'))
    df.update(zero_fill(train_df,'RAM'))
    df.update(zero_fill(train_df,'CPU'))
    df.update(zero_fill(train_df,'Internal storage'))
    
    df.update(fill_mean(train_df,df,'Days'))
    df.update(fill_mean(train_df,df,'screen size'))
    df.update(fill_mean(train_df,df,'resolution'))
    df.update(fill_mean(train_df,df,'Battery capacity'))
    df.update(fill_mean(train_df,df,'Price'))
    return df
val_df=full_fill(train_df,val_df)
test_df=full_fill(train_df,test_df)

C:\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
def one_col(df_train):# add x_0 - the column contains all ones
    df=df_train
    one=[1 for i in range(len(df))]
    df.insert(0,'one',one)
    return df
train_df=one_col(train_df)
val_df=one_col(val_df)
test_df=one_col(test_df)

In [14]:
train_df

,one,resolution,screen size,rear camera,front camera,CPU,RAM,Internal storage,Battery capacity,Days,Price,Oppo,Samsung,Sony,iPhone
0,1,2.0736e+06,5.2,13,5,1.6,2,16,2900,365.000000,6790000,0.0,1.0,0.0,0.0
1,1,1.22485e+06,4.70919,0,0,0,0,0,2561.5,197.853659,10490000,0.0,1.0,0.0,0.0
2,1,2.0736e+06,5.5,12,7,2.3,3,32,2900,134.000000,21999000,0.0,0.0,0.0,1.0
3,1,2.0736e+06,5.5,12,7,2.3,3,128,2900,3.000000,23999000,0.0,0.0,0.0,1.0
4,1,921600,5.0,8,5,0,2,16,3000,18.000000,2990000,0.0,0.0,0.0,0.0
5,1,409920,5,5,5,1.3,2,8,2400,197.853659,1790000,0.0,0.0,0.0,0.0
6,1,2.0736e+06,5.2,23,13,2.15,3,64,2900,270.000000,10990000,0.0,0.0,1.0,0.0
7,1,921600,5.0,13,5,0,2,16,2400,134.000000,4490000,0.0,1.0,0.0,0.0
8,1,921600,5.7,13,5,1.2,1,8,3000,134.000000,2490000,0.0,0.0,0.0,0.0
9,1,921600,5,13,5,1.4,2,16,2650,60.000000,3690000,0.0,0.0,0.0,0.0


In [15]:
def x_y(df):
    '''temp=df.copy()
    price=temp['Price'].values
    for i in range(len(price)):
        price[i]=re.sub('\.','',str(price[i]))
        print (price[i])
    price=price.astype(float).reshape(-1,1)
    temp=temp.drop('Price',axis=1)
    x=np.array(temp.values.astype(float))
    y=price'''
    temp=df.copy()
    y=temp['Price'].astype(float)
    temp=temp.drop('Price',axis=1)
    x=np.array(temp.values.astype(float))
    
    return x,y
x_train,y_train=x_y(train_df)
x_val,y_val=x_y(val_df)
x_test,y_test=x_y(test_df)
#train_df['Price'].astype('float')

In [16]:
#minmax normalization
x_train[:,1:]=preprocessing.minmax_scale(x_train[:,1:],axis=0)
x_test[:,1:]=preprocessing.minmax_scale(x_test[:,1:],axis=0)
x_val[:,1:]=preprocessing.minmax_scale(x_val[:,1:],axis=0)
y_train=y_train/10000000

In [17]:
X = x_train
Y = y_train

# Create neuron network model

In [18]:
numpy.random.seed(7)
# create model
def create_model(decay,lr):
    model = Sequential()
    model.add(Dense(128, input_dim=14, activation='sigmoid'))
    model.add(Dense(128, activation='sigmoid'))
    model.add(Dense(1,))
    #sgd = optimizers.SGD(lr=0.001, decay=0.000001, momentum=0.9, nesterov=True)
    sgd = optimizers.SGD(lr=lr, decay=decay, momentum=0.9, nesterov=True)
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])
    return model
def combination(a):
    r=[[]]
    for x in a:
        r = [ i + [y] for y in x for i in r ]
    return r
#model=create_model(0.00001,0.000001)
# Fit the model
#model.fit(X, Y, epochs=100, batch_size=10,verbose=2,validation_data=(x_val,y_val/10000000))
# evaluate the model
#scores = model.evaluate(X, Y)


In [21]:
decays=np.array([0.001,0.0001,0.00001,0.000001,0])
lrs=np.array([0.1,0.01,0.001,0.0001])
a = [[0,1,2,3,4],[0,1,2,3]]
val_error=[]
train_error=[]
pos=[]
combine=combination(a)
print ('')
for ele in combine:
    model=create_model(decays[ele[0]],lrs[ele[1]])
    a=model.fit(X, Y, epochs=1000, batch_size=50,verbose=0,validation_data=(x_val,y_val/10000000))
    
    val=a.history['val_loss']
    train=a.history['loss']
    
    #show graph
    plt.plot(range(1000), train,'b-', label='learning rate ')
    plt.plot(range(1000), val,'r-', label='decay coefficient ')
    plt.ylim([0,1])
    plt.title('lr: ' + str(lrs[ele[1]]) + ', decay co: ' + str(decays[ele[0]]))
    #plt.show()
    name='lr ' + str(lrs[ele[1]]) + ' decay co ' + str(decays[ele[0]]) + '.jpg'
    plt.savefig(name)
    plt.clf()
    
    val_scores = model.evaluate(x_val, y_val/10000000,verbose=0)
    train_scores=model.evaluate(X,Y,verbose=0)
    val_error.append(val_scores[0])
    pos.append([ele[0],ele[1]])
    print (decays[ele[0]],'\t',lrs[ele[1]],'\t',str(sum(val)/len(val)),'\t',str(sum(train)/len(train)),'\t',abs(sum(val)/len(val)-sum(train)/len(train)))


0.001 	 0.1 	 0.599296657676 	 1.1449014823 	 0.545604824624
0.0001 	 0.1 	 0.616251914757 	 0.516624122446 	 0.0996277923113
1e-05 	 0.1 	 0.40959079013 	 0.730964120436 	 0.321373330306
1e-06 	 0.1 	 0.375827810116 	 0.513523869837 	 0.137696059721
0.0 	 0.1 	 70626397.064 	 24286420.3886 	 46339976.6754
0.001 	 0.01 	 0.0958674964054 	 0.0674951561679 	 0.0283723402375
0.0001 	 0.01 	 0.117256476937 	 0.0677340320916 	 0.0495224448453
1e-05 	 0.01 	 0.109585434689 	 0.0677931122856 	 0.0417923224032
1e-06 	 0.01 	 0.119334220429 	 0.0690479869608 	 0.0502862334678
0.0 	 0.01 	 0.122824596167 	 0.0686406302237 	 0.0541839659435
0.001 	 0.001 	 0.265752121453 	 0.301202832036 	 0.0354507105822
0.0001 	 0.001 	 0.18917982681 	 0.211992074393 	 0.0228122475822
1e-05 	 0.001 	 0.18816800476 	 0.204537238792 	 0.0163692340316
1e-06 	 0.001 	 0.170991518096 	 0.190278058703 	 0.0192865406067
0.0 	 0.001 	 0.19921909572 	 0.222847268796 	 0.0236281730762
0.001 	 0.0001 	 0.36194646398 	 0.

Combine the current train dataset and the validation to be the bigger one for last training

In [ ]:
X=np.vstack((X,x_val))
Y=np.hstack((Y,y_val/10000000))

# Linear regression

In [ ]:
linear_model=LinearRegression()
linear_model.fit(X,Y)
y=linear_model.predict(X)
#print (sum(abs(y-y_test/10000000)))

sklearn.metrics.mean_squared_error(Y,y)


# Best neuron net

In [ ]:
def get_best(pos,error):
    mini=min(error)
    print (mini)
    ind=error.index(mini)
    return pos[ind]
best=get_best(pos,val_error)

In [ ]:
best_model=create_model(decays[best[0]],lrs[best[1]])
best_model.fit(X, Y, epochs=1000, batch_size=10,verbose=0)
scores = best_model.evaluate(x_test, y_test/10000000)
print (scores[0])

The output prices have been devided by 10 milions, in case you want to take the real values, multiply it by 10m

In [ ]:
pre=best_model.predict(x_test)
pre=pre.reshape((1,-1))
pre

In [ ]:
y_=(y_test/10000000).reshape((1,-1))
y_

In [ ]:
original=y_.reshape(-1)
prediction=pre.reshape(-1)
pan=pd.DataFrame()
pan['original']=original
pan['prediction']=prediction
pan

The error on average for each phone

In [ ]:
np.sum(np.abs(pre-y_))/60*10000000